### Programming for Science and Finance

*Prof. Götz Pfeiffer, School of Mathematical and Statistical Sciences, University of Galway*

# Notebook 7: Programming With Functions I

This notebook accompanies **Part II**. You will:

* understand how Python treats **functions as first-class objects**, and why this is useful
* see how **anonymous functions** and **closures** allow functions to capture and use surrounding state
* learn how to build and apply **decorators** to modify or enhance function behavior
* write decorators that handle **arbitrary arguments** using `*args` and `**kwargs`
* create **class-based decorators** that maintain internal state, such as for memoization
* use `functools.wraps` to preserve metadata when wrapping functions

These concepts form the basis for writing clear, reusable, and expressive Python code in scientific and financial applications.


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

## Task 1.  Function Attributes

Functions in Python are **first class objects**.  
As such they can be assigned ot variables,  passed as arguments into functions and also serve as return values.  
Like other objects, functions have **attributes** that can be accessed through the dot notation `obj.attr`.

**Example.** If we define a function `cube` with a **docstring** like this:

In [ ]:
def cube(x):
    """computes the cube of x"""
    return x * x * x

then the docstring becomes the function's `__doc__` attribute:

In [ ]:
cube.__doc__

and can be used by the online help operator `?`:

In [ ]:
?cube

A function has a name:

In [ ]:
cube.__name__

Most importantly, a function implements the special method `__call__` which makes it **callable**.  In some sense, `cube(3)` is just a shorthand for:

In [ ]:
cube.__call__(3)

---
**Exercises.**

1. What is the `type` of a function object, like `cube`?

2. The builtin python function `dir` lists all the attributes of an object.  How many attributes does the function  `cube` have?

---

## Task 2: Anonymous Functions.

Some functions don't need a name.  
An **anonymous function** can be defined by a **lambda expression** of the form
```
lambda params : expression
```
Here, `lambda` is a **keyword**, `params` is a list of names used as **parameters**, and `expression` is a Python **formula** for computing the output of the function.

Not every function can be written in this way.   But a function that is used only once and just computes the value of an expression is disposable and doesn't need a name.

In [ ]:
(lambda x : x**3)(3)

We might want to define a **function factory**, i.e., a function that makes and returns different functions, depending on an input parameter.

In [ ]:
def make_power(n):
    return lambda x : x**n

So, `make_power(2)` returns the function $x \mapsto x^2$, `make_power(3)` returns $x \mapsto x^3$, and so on.

In [ ]:
p3 = make_power(3)
p3(3)

We might want to define a functon `shifted` that returns a shifted variant of a function `f`, shifted vertically by `v`, and horizontally by `h`. 

In [ ]:
def shifted(f, v, h):
    return lambda x: f(x - h) + v

This is an example of a function that has a function as input and returns a function as output.  
For example, suppose `f` is the function $f(x) = \frac12(x^3 - x + 1)$.

In [ ]:
f = lambda x: (x**3 - x + 1)/2

We can compute and plot $f(x)$  for $x \in [-1.8, 1.1]$, and the same graph shifted vertically by $2$ or by $-1$, and shifted horizontally by $\frac12$ or by $-\frac12$ as follows.

In [ ]:
xxx = np.linspace(-1.8, 1.1, 50)
params = [(0,0), (2,0), (-1,0), (0,1/2), (0,-1/2)]
for v, h in params:
    plt.plot(xxx+h, [shifted(f, v, h)(x) for x in xxx+h])  

---
**Exercises.**

3. Define a function `evaluate` that takes a function `f` and a list `xxx` of $x$-values as argument, and returns the list of values $f(x)$ for $x$ in `xxx`.

---

##  Task 3. Loggers

Suppose we want to see a message whenever a function is called, perhaps with the list of arguments, and perhaps another message when it returns, and with which return value.

**Example.** The factorial function $f(n) = n!$, defined by
$$
n! = \begin{cases}
1, & \text{if } n = 0,\\
(n-1)!, & \text{else.}
\end{cases}
$$

In [ ]:
def factorial(n): 
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(5)

In [ ]:
def factorial1(n):
    print(f"calling factorial with argument {n}")
    value = 1 if n == 0 else factorial1(n-1) * n
    print(f"return {value}")
    return value

In [ ]:
factorial1(5)

It would be more elegant do this programmatically:

In [ ]:
def add_messages(f):
    def ff(x):
        print(f"calling {f.__name__} with argument {x}")
        value = f(x)
        print(f"return {value}")
        return value
    return ff
        

If we apply this to our `factorial` function:

In [ ]:
fm = add_messages(factorial)
fm(5)

... we only get a report on the outer most call.   
But if we rename the modified function to `factorial`:

In [ ]:
factorial = fm
factorial(5)

...  we get a full report on each intermediate call.

---
**Exercises.**

1. Write a function `add_counter(f)` that adds a counter to the given function `f`,
   i.e., that builds a new function `fc` say, that increments its counter `fc.count`
   before calling `f(x)`, and initializes `fc.count` to $0$ before returning it.
   
2. Apply `add_counter` to the `factorial` function, then compute `factorial(6)` and check how often the function has been called.
---

## Task 4. Decorators

Python has a special syntax for this kind of modification to a function's behavior: **decorators**.

In [ ]:
@add_messages
def factorial(n): 
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(5)

The code
```python
@decorated
def func():
    print("Hi!")
```
is equivalent to
```python
def func():
    print("Hi!")
func = decorated(func)
```
That's all there is to it.

However, if we apply `add_messages` in its current form to `gcd`:
```python
@add_messages
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)
```
a call to `gcd(60, 24)` will fail because `f(x)` inside `add_messages` is called with only one argument instead of `gcd`'s two arguments.

---
**Exercises.**

1. Apply `add_counter` as a decorator to the `factorial` function.  Call `factorial(7)` and check how often `factorial` has been called.

2. Use the definition of the Fibonacci numbers 
   $$
   F_n = \begin{cases}
   n, & \text{if } n < 2,\\
   F_{n-2} + F_{n-1}, & \text{else}
   \end{cases}
   $$
   to write a function `fibonacci(n)` that computes $F_n$.

3. Decorate the `fibonacci` function with `add_counter`, call `fibonacci(10)` and check
   how often `fibonacci` has been called

---

## Task 5. Multiple Arguments

Python uses a simple method to allow a function to have multiple arguments, without having to specify their exact number.

Recall **list unpacking**:  putting `*` in front of a list (or tuple), essentially removes
a pair of brackets (or parentheses), `*(1, 2, 3)` becomes `1, 2, 3` in a suitable context,
like the following:

In [ ]:
[0, *(1, 2, 3), 4]

In reverse, **list packing** adds a pair of brackets, when needed, for example in a parallel assignment:

In [ ]:
[a, *b, c] = [0, 1, 2, 3, 4]
print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")


Both operations can be used at the same time:

In [ ]:
data = [2, 3, 4]
[x, y, *z] = [1, *data, 5]
print(f"x = {x}")
print(f"y = {y}")
print(f"z = {z}")


In a function definition, the **last** parameter can be a list packing parameter, often called `*args`.  
For context, let's briefly look at how the **arguments** given in a **function call** are matched up with the **parameters** declared in a **function definition**, either by their **position** or by a **keyword**.

### Function Call

In Python, a typical function call has the form
```python
value = f(1, 2, 3, ..., e=2, d=2, ...)
```
Here the arguments `1, 2, 3, ...` are **positional** arguments: they are matched up by their position in the argument list, with the parameter at the same position in the function definition.  
In contrast, the arguments `e=2, d=2, ...` are **keyword** arguments:  they
are matched by the keyword with the parameter of the same name in the function definition.
(It therefore doesn't matter in which order they are listed.)  
In any case, the positional arguments must come before the keyword arguments: they cannot be mixed.


### Function Definition

In Python, a typical function definition head has the form
```python
def f(a, b, xc, ..., d=0, e=1, ...):
```
Here, the parameters `a, b, c, ...` are **required** parameters: values for them need to be provided as arguments in the function call.   
In contrast, the parameters `d, e, ...` are **optional** parameters equipped with **default values** `d=0, e=1, ...`. If the function call provides a value for an optional parameter, this value overrides the default value.
In any case, the required parameters must come before the optional parameters: they cannot be mixed.

A function definition can use argument unpacking to collect several positional arguments in a list `args`, and several keyword arguments in a dictionary `kwargs` by including their starred versions in the parameter list:
```python
def f(..., *args, ..., **kwargs):
```
If a function is defined in this way, the parameters before `*args` can only receive
positional arguments.  After matching these up, all the remaining positional arguments
are collected ad a list in `args`.  The parameters between `*args` and `**kwargs` can only
receive keyword arguments.  After matching these up, all remaining keyword arguments are 
collected in a dictionary `kwargs`.

Here `args` and `kwargs` are just the names used by convention.

If applied correctly, this can be a powerful way to add flexibility to your function calls.

### Decorators for Functions with Multiple Arguments

For example, when decorating a function with an unknown number of parameters.
As we don't know in general, how many arguments the decorated function
`f` will have, we simply wrap them all up in a list `args`.  
We use **list packing** in the definition of the new function `ff`
and **list unpacking** when we pass these arguments into the original function `f`.
Note how the syntax for both processes (which are reverse to each other) is the same.

In [ ]:
def add_messages(f):
    def ff(*args, **kwargs):
        print(f"calling {f.__name__} with arguments {args} and {kwargs}")
        value = f(*args, **kwargs)
        print(f"return {value}")
        return value
    return ff

In this form, we can apply the decorator to `factorial` (with 1 argument)
and to `gcd` (with 2 arguments).

In [ ]:
@add_messages
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60, 24)

---
**Exercises.**

1. Some function definition heads contain the symbols `/` or `*` as explicit parameters,
   i.e., they look like
   ```python
   def f(..., /, ..., *, ...):
   ```
   Consult the documentation to find out what that means.

2.  Define a silly `demo` function as
    ```python
    def demo(a, b, /, c, *, d):
        return a, b, c, d
    ``` 
    Then try
    ```python
    demo(1, 2, 3, d=4)
    demo(a=1, b=2, c=3, d=4)
    demo(1,2,c=3,4)
    ```
    and explain the resulting error messages, if any.

2. Modify the decorator function `add_counter` so that it can be applied to functions with multiple arguments.
---

## Task 6.  Decorator Classes

A decorator need not be a function, it just has to be **callable**.  
An object `obj` of a class is callable if the class implements the special method `__call__`.
Then `obj(x)` can be used as a shorthand for `obj.__call__(x)`.

We now define a class based decorator `Memoize`, whose purpose is to remember return values of function calls.  In the case of a function that recursively calls itself, reusing previously computed return values can drastically reduce the number of necessary calculations.


An object `m` of this class will have the undecorated function as attribute `m.func`.  
The attribute `m.cache` will be a (growing) dictionary of key-value pairs
of the form `args : value`.  
Actually, we normalize the `kwargs` part of the argument list to avoid unnecessary duplication.  
Decoration of a function `f` will work in the ususal way:
```python
@Memoize
def f(x):
    print("Hi")
```
is again equivalent to
```python
def f(x):
    print("Hi")
f = Memoize(f)
```

In [ ]:
class Memoize:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        if key not in self.cache:
            print(f"calling {self.func.__name__} with arguments {args} and {kwargs}")
            self.cache[key] = self.func(*args, **kwargs)
            print(f"return {self.cache[key]}")
        return self.cache[key]

We apply the decorator to the Fiboncci function:

In [ ]:
@Memoize
def fib(n): return n if n < 2 else fib(n-1) + fib(n-2)


... and observe how it drastically reduces the number of calls:

In [ ]:

print(fib(10))  # 55
print(fib.cache)  # inspect cache

---
**Exercises.**

1. Write a class based decorator `Count` that counts how often a decorated function is called.

2. Add a `clear()` method to the `Count` class that can be used to reset a function's counter to $0$.
---

## Summary



This notebook introduced several core ideas about Python’s treatment of functions:

* **Functions as first-class objects:** they can be assigned to variables, stored in containers, and passed or returned like any other value.
* **Anonymous functions (lambdas):** compact ways to define small functions.
* **Decorators:** higher-order functions that wrap and modify other functions, enabling patterns such as logging, validation, and transformation of behavior.
* **General argument handling:** use of `*args` and `**kwargs` to write flexible decorators that accept any function signature.
* **Class-based decorators:** using objects with a `__call__` method to implement decorators that maintain internal state (e.g., memoization).

Together, these concepts illustrate how Python’s functional features support clear, expressive, and extensible program design.
